In [1]:
import pandas as pd
import requests
import time
import os
from tqdm import tqdm

# 🚨 Substitua por suas credenciais Spotify
CLIENT_ID = "d340981fec2c4ef69d7c18f3718ea66b"
CLIENT_SECRET = "44543e32c9794347a336d2675b518d73"

# Autenticação com Client Credentials Flow
def autenticar_spotify(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    resp = requests.post(url, {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    })
    resp.raise_for_status()
    return resp.json()["access_token"]

# Consulta até 50 faixas por chamada
def buscar_tracks_em_lote(ids, token):
    url = "https://api.spotify.com/v1/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"ids": ",".join(ids)}
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()["tracks"]

# Caminho da pasta com os CSVs de lotes (entrada) e pasta de destino (saída)
pasta_lotes = r"C:\Users\Lucas\Documents\Coding\analise_duracao_spotify\data\raw\blocos de id"
pasta_saida = r"C:\Users\Lucas\Documents\Coding\analise_duracao_spotify\data\raw"
os.makedirs(pasta_saida, exist_ok=True)

# Autentica e obtém token
token = autenticar_spotify(CLIENT_ID, CLIENT_SECRET)

# Lista de todos os arquivos CSV na pasta
arquivos = sorted([
    os.path.join(pasta_lotes, f) for f in os.listdir(pasta_lotes)
    if f.endswith(".csv")
])

# Lista para armazenar os resultados
registros = []

# Loop por cada arquivo CSV
for arquivo in tqdm(arquivos, desc="Consultando lotes"):
    df_ids = pd.read_csv(arquivo)
    ids = df_ids["track_id"].dropna().tolist()

    try:
        tracks = buscar_tracks_em_lote(ids, token)
        for track in tracks:
            if track is None:
                continue
            registros.append({
                "track_id": track["id"],
                "name": track["name"],
                "artists": ", ".join([a["name"] for a in track["artists"]]),
                "album": track["album"]["name"],
                "release_date": track["album"]["release_date"],
                "duration_ms": track["duration_ms"],
                "explicit": track["explicit"],
                "popularity": track["popularity"],
                "preview_url": track["preview_url"],
                "spotify_url": track["external_urls"]["spotify"]
            })
        time.sleep(0.2)  # Evita rate limiting
    except Exception as e:
        print(f"Erro no arquivo {arquivo}: {e}")

# Converte para DataFrame final
df_final = pd.DataFrame(registros)

# Caminho completo para o arquivo de saída
arquivo_saida = os.path.join(pasta_saida, "spotify_tracks_detalhes.csv")
df_final.to_csv(arquivo_saida, index=False)
print(f"✅ Arquivo final salvo: {arquivo_saida}")


Consultando lotes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [01:11<00:00,  1.55it/s]

✅ Arquivo final salvo: C:\Users\Lucas\Documents\Coding\analise_duracao_spotify\data\raw\spotify_tracks_detalhes.csv
